In [2]:
import sys
sys.path.append('../')
import json
import pandas as pd
import sqlite3
from helper import load_dotenv
from tqdm.auto import tqdm

In [4]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

# BackUp ziehen

In [ ]:
0 / 0

In [ ]:
data.to_pickle(dotenv['TEMP_PROCESSING_DIR'] / '24-04-28_trapezoidal_distortion_backup.pkl')

# Update

In [5]:
data = pd.read_sql(
    '''
        SELECT * FROM adjustment_per_mask apm 
        WHERE apm.adjustment = 'trapezoidal_distortion'
    ''',
    con
)

In [ ]:
updates = []

for val in data.features:
    features = json.loads(val)
    if pd.isna(features['trapezoidal_distortion_strength_1']):
        features['trapezoidal_distortion_strength_1'] = 0

    if pd.isna(features['trapezoidal_distortion_strength_2']):
        features['trapezoidal_distortion_strength_2'] = 0
    
    if features['trapezoidal_distortion_direction'] == 'top':
        corner_adjustments = [
            (features['trapezoidal_distortion_strength_1'],0),
            (0,0),
            (0,0),
            (features['trapezoidal_distortion_strength_2'],0)
        ]
    elif features['trapezoidal_distortion_direction'] == 'left':
        corner_adjustments = [
            (features['trapezoidal_distortion_strength_2'],0),
            (features['trapezoidal_distortion_strength_1'],0),
            (0,0),
            (0,0)
        ]
    elif features['trapezoidal_distortion_direction'] == 'bottom':
        corner_adjustments = [
            (0,0),
            (features['trapezoidal_distortion_strength_2'],0),
            (features['trapezoidal_distortion_strength_1'],0),
            (0,0)
        ]
    else:
        corner_adjustments = [
            (0,0),
            (0,0),
            (features['trapezoidal_distortion_strength_1'],0),
            (features['trapezoidal_distortion_strength_2'],0)
        ]
    
    updates.append(json.dumps({ 'corner_adjustments' : corner_adjustments }))

In [ ]:
len(updates), data.shape

In [ ]:
data.iloc[10]

In [ ]:
c = con.cursor()

for i in tqdm(range(len(updates))):
    row = data.iloc[i]
    
    c.execute(
        f'''
            UPDATE adjustment_per_mask
            SET features='{ updates[i] }'
            WHERE job='{ row.job }' AND pdf_filename='{ row.pdf_filename }' AND mask_id='{ row.mask_id }' AND adjustment='{ row.adjustment }'
        '''
    )

c.close()
con.commit()

# Update Uniform_trap

In [6]:
data_uniform = pd.read_sql(
    '''
        SELECT * FROM adjustment_per_mask apm 
        WHERE apm.adjustment = 'uniform_trapezoidal_distortion'
    ''',
    con
)

In [23]:
c = con.cursor()

for i in tqdm(range(data_uniform.shape[0])):
    row = data_uniform.iloc[i]
    features = json.loads( row.features )

    if type(features['trapezoidal_distortion_strength']) == float:
        features['trapezoidal_distortion_strength'] = (features['trapezoidal_distortion_strength'],0)
    
        sql = f'''
            UPDATE adjustment_per_mask
            SET features='{ json.dumps(features) }'
            WHERE job='{ row.job }' AND pdf_filename='{ row.pdf_filename }' AND mask_id='{ row.mask_id }' AND adjustment='{ row.adjustment }'
        '''
        c.execute(sql)

c.close()
con.commit()

  0%|          | 0/2794 [00:00<?, ?it/s]

# Update wave_deform

In [5]:
data_wave_deform = pd.read_sql(
    '''
        SELECT * FROM adjustment_per_mask apm 
        WHERE apm.adjustment = 'wave_deform'
    ''',
    con
)

In [19]:
c = con.cursor()

for i in tqdm(range(data_wave_deform.shape[0])):
    row = data_wave_deform.iloc[i]
    features = json.loads( row.features )

    if 'wave_overlay_count' not in features:
        features = {
            'wave_overlay_count' : 1,
            'wave_configurations' : [[features['wave_length'],features['wave_depth']]]
        }
        #features['trapezoidal_distortion_strength'] = (features['trapezoidal_distortion_strength'],0)
    
        sql = f'''
            UPDATE adjustment_per_mask
            SET features='{ json.dumps(features) }'
            WHERE job='{ row.job }' AND pdf_filename='{ row.pdf_filename }' AND mask_id='{ row.mask_id }' AND adjustment='{ row.adjustment }'
        '''
        c.execute(sql)

c.close()
con.commit()

  0%|          | 0/3686 [00:00<?, ?it/s]